# In this cell, we will perform Data Cleaning

In [1]:
#Import Statements
import numpy as np
import math
import matplotlib.pyplot as plt
import torchvision
import torch
from torchvision import datasets, transforms
from torch.utils.data import random_split
import torch.nn as nn
import torchvision.models as models
from tqdm import tqdm
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [2]:
#First, we define our data loader. 
#Much of this is from HW6
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y
    def __len__(self):
        return len(self.subset)
def prepare_and_get_loaders(validation_split, batch_size, num_workers, dir_path):
    """
    This function loads and partitions our image data.
    
    val_split: proportion of the data that will be in the validation split
    batch_size: size of batches
    num_workers: number of parallel processors (-1 to use all)
    dir_path: directory path of the data
    """
    
    #Transforms to apply to training and test data
    #Useful augmentation: transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    data_transforms = {
        'train': transforms.Compose([
        transforms.Resize((244, 244)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) 
    ]),

        'test': transforms.Compose([
            transforms.Resize((244,244)),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])
    }
    full_dataset = datasets.ImageFolder(dir_path)
    
    m = len(full_dataset)
    val_size = int(validation_split * m)
    train_size = m - val_size
    
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
    train_dataset = MyDataset(train_dataset, transform=data_transforms['train'])
    val_dataset = MyDataset(val_dataset, transform=data_transforms['test'])
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    
    return train_loader, val_loader

In [3]:
def finetune(model, train_loader, val_loader, learning_rate, num_epochs, L2, momentum1, momentum2, stepsize, gamma):
    """

    Finetunes the given model on the loader data, and returns the trained model as well as the losses
    and final validation accuracy for plotting. 
    
    model: the Pytorch model
    train_loader: the loader for the training data
    val_loader: the loader for the validation data
    learning_rate: alpha, the learning rate
    num_epochs: the number of epochs to train for
    L2: the L2 regularization value
    momentum1: the first momentum value
    momentum2: the second momentum value
    stepsize: the number of epochs between lowering the learning rate
    gamma: the ratio to multiply the learning rate at each step size

    """
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"My Device is {device}")
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=L2, betas = (momentum1, momentum2))
    
    scheduler = StepLR(optimizer, step_size=stepsize, gamma=gamma)

    hist = {'train': [], 'val': []} # History of training and validation losses

    for epoch in tqdm(range(num_epochs), desc = "Training Epoch: "):
        model.train() #Training mode
        total_loss = 0.0
        for inputs, labels in train_loader:
            #put stuff on device
            inputs = inputs.to(device)
            labels = labels.to(device)

            #zero optimizer
            optimizer.zero_grad()

            #get outputs + train
            outputs = model(inputs)

            #get loss and do backward
            loss = criterion(outputs, labels)
            loss.backward()

            #step
            optimizer.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        hist['train'].append(avg_train_loss)

        #Validation Mode Now
        model.eval()
        total_loss = 0.0

        #keep track of correct labels so we can get the val accuracy
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                #put on device
                inputs = inputs.to(device)
                labels = labels.to(device)

                #get outputs
                outputs = model(inputs)

                #get loss
                loss = criterion(outputs, labels)
                total_loss += loss.item()

                #https://www.digitalocean.com/community/tutorials/pytorch-torch-max
                max_element, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        avg_val_loss = total_loss / len(val_loader)
        hist['val'].append(avg_val_loss)
        val_accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

    #final val accuracy
    final_val_accuracy = 100*correct/total

    return model, hist, final_val_accuracy

In [6]:
def load_model(arch, dropout, num_classes):
    """

    Loads a pretrained model, but with the final output layer removed and replaced with a randomly
    initialized output layer that will classify among the 5 buildings.

    """
    
    #select the model we want
    if arch == "resnet18": 
        model = models.resnet18(weights="ResNet18_Weights.DEFAULT")
        num_ftrs = model.fc.in_features
        model.fc = nn.Sequential(nn.Dropout(dropout), nn.Linear(num_ftrs, num_classes)) 
    elif arch == "mobile":
        model = models.mobilenet_v2(weights="MobileNet_V2_Weights.DEFAULT")
        num_ftrs = model.classifier[1].in_features
        model.classifier = nn.Sequential(nn.Dropout(dropout), nn.Linear(num_ftrs, num_classes))
    else:
        print("Incorrect model Architecture selected")
        raise
    
    return model

In [ ]:
#Dataset parameters
validation_split = 0.2
batch_size = 16
num_workers = -1
dir_path = 'Data_400'

train_loader, val_loader = prepare_and_get_loaders(validation_split, batch_size, num_workers, dir_path):

In [7]:
#Model Parameters
arch = "mobile"
dropout = 0.3
num_classes = 5
model = load_model(arch, dropout, num_classes)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to C:\Users\Brandon/.cache\torch\hub\checkpoints\mobilenet_v2-7ebf99e0.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [ ]:
#Finetuning Parameters
learning_rate = 0.0001
num_epochs = 10
L2 = 10**(-5)
momentum1 = 0.9
momentum2 = 0.99
stepsize = 5
gamma = 0.1
model, hist, final_val_accuracy = finetune(model,
                                          train_loader,
                                          val_loader,
                                          learning_rate,
                                          num_epochs,
                                          L2,
                                          momentum1,
                                          momentum2, 
                                          stepsize,
                                          gamma)
print(f"Our final validation accuracy accuracy using the full CNN is {final_val_accuracy:.2f}%")

In [ ]:
def plot_history(hist):
    epochs = range(1, len(hist['train']) + 1)

    # Plot training and validation loss
    plt.figure(figsize=(12, 6))
    plt.plot(epochs, hist['train'], 'b', label='Training loss')
    plt.plot(epochs, hist['val'], 'r', label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

In [ ]:
plot_history(hist)